# 양방향LSTM model

In [1]:
import pandas as pd 
import pandas_datareader as pdr   
import numpy as np
from sklearn.preprocessing import StandardScaler



In [2]:
import torch
import random
import os
random.seed(123)
np.random.seed(123)
os.environ["PYTHONHASHSEED"] = str(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)               # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True      # type: ignore

## 1. 데이터 로드 및 전처리

In [6]:
final2 = pd.read_csv('/home/jbj4278/ETH_data/final_dateadd') #feature포함 데이터

In [7]:
col = ['Date','Open', 'High', 'Low', 'Close', 'Volume','BTC_close', 'DXY', 'BTCD', 'Kimchi_premium', 'S&P500', 'Ethereum DeFi',
   'News_freq','signal']

In [8]:
final = final2.loc[:,col]

In [9]:
final

,Date,Open,High,Low,Close,Volume,BTC_close,DXY,BTCD,Kimchi_premium,S&P500,Ethereum DeFi,News_freq,signal
0,2021-03-23 22:09:00,2019000,2020000,2017000,2018000,101.666675,65987000.0,92.350,61.201411,10.897713,3910.51,4.492662e+10,142.0,0.0
1,2021-03-23 22:10:00,2018000,2019000,2017000,2019000,20.106467,65985000.0,92.350,61.201411,10.897713,3910.51,4.492662e+10,142.0,0.0
2,2021-03-23 22:11:00,2019000,2020000,2017000,2019000,17.313993,66057000.0,92.350,61.201411,10.897713,3910.51,4.492662e+10,142.0,0.0
3,2021-03-23 22:12:00,2020000,2020000,2019000,2019000,36.582210,66012000.0,92.350,61.201411,10.897713,3910.51,4.492662e+10,142.0,0.0
4,2021-03-23 22:13:00,2020000,2020000,2016000,2016000,63.727452,66015000.0,92.350,61.201411,10.897713,3910.51,4.492662e+10,142.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523096,2022-03-23 15:15:00,3666000,3667000,3665000,3665000,39.688187,51801000.0,98.793,42.584513,15.249800,4488.22,1.159212e+11,57.0,0.0
523097,2022-03-23 15:16:00,3666000,3669000,3665000,3668000,76.731647,51753000.0,98.793,42.584513,15.249800,4488.22,1.159212e+11,57.0,0.0
523098,2022-03-23 15:17:00,3668000,3671000,3667000,3669000,96.039465,51782000.0,98.793,42.584513,15.249800,4488.22,1.159212e+11,57.0,0.0
523099,2022-03-23 15:18:00,3667000,3674000,3663000,3663000,91.949217,51780000.0,98.793,42.584513,15.249800,4488.22,1.159212e+11,57.0,0.0


In [10]:

final.loc[final['signal']==0.0 , "signal"] = 2   # 보합 > 2로
final.loc[final['signal']==1.0 , "signal"] = 0  
 # 상승 > 0으로
final.loc[final['signal']==-1.0 , "signal"] = 1 #하락 > 1로
#(상승, 하락, 보합) > (0 ,1, 2)

In [11]:
final['signal'].value_counts()


0.0    188025
1.0    171867
2.0    163209
Name: signal, dtype: int64

In [12]:
def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [13]:
len_test = int(len(final)*0.2)
test_set = final[-len_test:]
tr_set = final[:-len_test]

In [14]:
test_set.shape

(104620, 14)

In [15]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
import wandb
!wandb login

wandb: Currently logged in as: gyeongmocho. Use `wandb login --relogin` to force relogin


In [17]:
from torchmetrics.functional import f1_score,precision_recall




In [18]:
import time

def train(dataloader):
 
    model.train()
    total_acc, total_count = 0, 0
    start_time = time.time()

    
    loss_list = []
    for idx, (feature, label) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(feature.to(device))
        loss = criterion(predicted_label, label.to(device))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        
        total_count += label.size(0)
        loss_list.append(loss.item())
        
        if idx+1 % len(feature) == 0 and idx > 0:
            accuracy = total_acc/total_count
            loss_hist = np.mean(loss_list)
            elapsed = time.time() - start_time
            print(f'| epoch {epoch} | {idx}/{len(dataloader)} batches '
                f'| accuracy {accuracy} | loss {loss_hist}')
            total_acc, total_count = 0, 0
            start_time = time.time()

    wandb.log({'train_loss':np.mean(loss_list),'train_acc':total_acc/total_count})  
    return  total_acc/total_count,np.mean(loss_list)
    
            

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    loss_list = []
    
    score,precision,recall = 0, 0, 0
    with torch.no_grad():
        for idx, (feature, label) in enumerate(dataloader):
            feature, label = feature.to(device),label.to(device)
            predicted_label = model(feature.to(device))
            loss = criterion(predicted_label, label.to(device))
            
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            loss_list.append(loss.item())
            
            
            
            score += f1_score(predicted_label.argmax(1).to(device), label,num_classes=3)
            precision += precision_recall(predicted_label.argmax(1).to(device), label, average='macro', num_classes=3)[0]
            recall += precision_recall(predicted_label.argmax(1).to(device), label, average='macro', num_classes=3)[1]
      
        
        score = score / len(loss_list)
        precision = precision / len(loss_list)
        recall = recall / len(loss_list)
            
    wandb.log({'val_loss':np.mean(loss_list), 'val_acc':total_acc/total_count,'f1_score':score,
              'precision':precision,'recall':recall})
    
    return total_acc/total_count, np.mean(loss_list),score




            
   

In [32]:
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self,path, patience=10, verbose=False, delta=0,):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [20]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)
    
        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.9 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]


## 모델정의

In [21]:
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiLSTM, self).__init__() # 상속한 nn.Module에서 RNN에 해당하는 init 실행
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=True,batch_first=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        

    def forward(self, x): 
        # input x : (BATCH, LENGTH, INPUT_SIZE) 
        # 최초의 hidden state와 cell state를 초기화
        # 만약 Bi-directional LSTM이라면 아래의 hidden and cell states의 첫번째 차원은 2*self.num_layers 
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # (BATCH SIZE, SEQ_LENGTH, HIDDEN_SIZE)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)  # hidden state와 동일

        # LSTM 순전파
        out, (hn, cn) = self.lstm(x, (h0, c0)) # out : (BATCH_SIZE, SEQ_LENGTH, HIDDEN_SIZE)  , hn이 lstm의 마지막 hidden state
        
        
        
        
        out = self.fc(out[:, -1, :])
        
        
        return out

In [22]:
window_size = 10

In [23]:
# parameters
input_size = 12
hidden_size = 20
num_layers = 1
num_classes = 3
lr = 0.0001


## 모델학습

In [48]:
btss = BlockingTimeSeriesSplit(n_splits=5)
n_epochs = 100
scale_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'BTC_close', 'DXY', 'BTCD',
       'Kimchi_premium', 'S&P500', 'Ethereum DeFi', 'News_freq']
score_list = []
val_loss_list = []
voting_list = []
#cross validation

wandb.init(project='BiLSTM', entity='gyeongmoCho',name='BiLSTM_2lay_smallbatch')
#wandb.watch(model, criterion, log = "all" )


for idx,(tr_idx, val_idx) in enumerate(btss.split(tr_set)):
    
    train_data, val_data = tr_set.iloc[tr_idx], tr_set.iloc[val_idx]
    
    
    scaler = StandardScaler()
    scaler.fit(train_data[scale_cols])#for문을 돌면서 각 train, val을 스케일링ㅇ
    train_feature = scaler.transform(train_data[scale_cols])
    train_feature = pd.DataFrame(train_feature)                                 
    val_feature = scaler.transform(val_data[scale_cols])
    val_feature = pd.DataFrame(val_feature)   

    
    train_feature, train_label = make_dataset(train_feature, train_data['signal'], window_size)
    val_feature, val_label = make_dataset(val_feature, val_data['signal'], window_size)

    
    #모델 선언 
    
    model = BiLSTM(input_size,hidden_size, num_layers,num_classes).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0,verbose=True)
    total_accu = None


    print(model)
    
    early_stopping = EarlyStopping(patience = 10, verbose = True)



    #스케일링된 데이터를 torch Tensor로 변경
    val_feature = torch.FloatTensor(val_feature).to(device)
    val_label = torch.LongTensor(val_label).to(device)
    train_feature = torch.FloatTensor(train_feature).to(device)
    train_label  = torch.LongTensor(train_label).to(device)

    train_set = TensorDataset(train_feature,train_label)
    val_set = TensorDataset(val_feature,val_label)
    
    #데이터로더 사용
    BATCH_SIZE = 32
    train_dataloader = DataLoader(train_set , batch_size=BATCH_SIZE,shuffle=False)
    valid_dataloader = DataLoader(val_set, batch_size=BATCH_SIZE,shuffle=False)

    print(f"train_feature:{train_feature.shape},val_feature:{val_feature.shape}" )
    

    for epoch in range(1, n_epochs + 1):
        # 현재 learning rate 출력
        print(optimizer.param_groups[0]['lr'])
        epoch_start_time = time.time()
        accu_train, train_loss = train(train_dataloader)
        accu_val,val_loss,score = evaluate(valid_dataloader)
        if total_accu is not None and total_accu > accu_val:
            scheduler.step()
        else:
            total_accu = accu_val
        print('-' * 59)
        print(f'| end of epoch {epoch} | time: {time.time()- epoch_start_time}s | '
            f'valid accuracy {accu_val} | valid loss {val_loss}'
            ) 

        print('-' * 59)
        
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    score_list.append(score)
    print("save score")
    #voting_prob = soft_voting(valid_dataloader)
    #voting_list.append(voting_prob)
    print("save voting prob")
    
    PATH = f"/home/jbj4278/ETH_data/BiLSTM_2lay_smallbatch{idx}.pt"
    torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),


                }, PATH)
    print('-' * 59)
    print('Fold Finish')
    print('-' * 59)

score_list = list(map(float, score_list))
final_f1_score = np.mean(score_list)
print(final_f1_score)
wandb.log({'mean_f1_score':final_f1_score})



        
    

   

f1_score,▃▃▃▆▇▇▇▇▇▇▇▇▇▄▂▁▁▂▃▄▆▆▆▇▇▇▇▇▇▆▆▆▇▇▇▇▇▇▇█
mean_f1_score,▁
precision,▂▂▃▇▇▇▇▇▇▇▇▇▇▃▁▁▁▂▃▄▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██
recall,▃▃▃▇▇▇▇▇▇▇▇▇▇▂▂▁▁▂▂▃▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇█
train_acc,▁▁▅▅▆▆▆▆▆▆▆▆▆▄▅▃▃▃▃▄▆▆▆▆▅▆▆▆▆▇██▆▆▆▆▆▆▆▆
train_loss,█▄▁▅▅▄▄▄▄▃▃▃▃▅▄▆▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▅▅▄▄▄▄▄▃
val_acc,▃▃▃▆▇▇▇▇▇▇▇▇▇▄▂▁▁▂▃▄▆▆▆▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇█
val_loss,▅▆█▃▃▂▂▂▂▂▂▂▂▄▅▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁
f1_score,0.52636
mean_f1_score,0.41312
precision,0.49962


Adjusting learning rate of group 0 to 1.0000e-04.
BiLSTM(
  (lstm): LSTM(12, 20, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=40, out_features=3, bias=True)
)
train_feature:torch.Size([75316, 10, 12]),val_feature:torch.Size([8360, 10, 12])
0.0001
-----------------------------------------------------------
| end of epoch 1 | time: 11.863046407699585s | valid accuracy 0.4777511961722488 | valid loss 1.0886858574306693
-----------------------------------------------------------
Validation loss decreased (inf --> 1.088686).  Saving model ...
0.0001
Adjusting learning rate of group 0 to 9.9975e-05.
-----------------------------------------------------------
| end of epoch 2 | time: 12.011900186538696s | valid accuracy 0.3757177033492823 | valid loss 1.0973013452901186
-----------------------------------------------------------
EarlyStopping counter: 1 out of 10
9.997532801828658e-05
Adjusting learning rate of group 0 to 9.9901e-05.
-------------------------

-----------------------------------------------------------
| end of epoch 14 | time: 11.760196924209595s | valid accuracy 0.4717703349282297 | valid loss 1.086842299417685
-----------------------------------------------------------
Validation loss decreased (1.088180 --> 1.086842).  Saving model ...
0.0001
-----------------------------------------------------------
| end of epoch 15 | time: 11.739846467971802s | valid accuracy 0.4717703349282297 | valid loss 1.085600246682422
-----------------------------------------------------------
Validation loss decreased (1.086842 --> 1.085600).  Saving model ...
0.0001
-----------------------------------------------------------
| end of epoch 16 | time: 11.751567840576172s | valid accuracy 0.4717703349282297 | valid loss 1.0844461451504976
-----------------------------------------------------------
Validation loss decreased (1.085600 --> 1.084446).  Saving model ...
0.0001
-----------------------------------------------------------
| end of epo

-----------------------------------------------------------
| end of epoch 41 | time: 11.569612979888916s | valid accuracy 0.4717703349282297 | valid loss 1.0713248746540711
-----------------------------------------------------------
Validation loss decreased (1.071527 --> 1.071325).  Saving model ...
0.0001
-----------------------------------------------------------
| end of epoch 42 | time: 11.773584604263306s | valid accuracy 0.4717703349282297 | valid loss 1.0711371086480963
-----------------------------------------------------------
Validation loss decreased (1.071325 --> 1.071137).  Saving model ...
0.0001
-----------------------------------------------------------
| end of epoch 43 | time: 11.64967656135559s | valid accuracy 0.4717703349282297 | valid loss 1.070962916121228
-----------------------------------------------------------
Validation loss decreased (1.071137 --> 1.070963).  Saving model ...
0.0001
-----------------------------------------------------------
| end of epo

-----------------------------------------------------------
| end of epoch 68 | time: 11.816051721572876s | valid accuracy 0.4717703349282297 | valid loss 1.0695914555596941
-----------------------------------------------------------
EarlyStopping counter: 5 out of 10
0.0001
-----------------------------------------------------------
| end of epoch 69 | time: 11.746776819229126s | valid accuracy 0.4717703349282297 | valid loss 1.0696214034812142
-----------------------------------------------------------
EarlyStopping counter: 6 out of 10
0.0001
-----------------------------------------------------------
| end of epoch 70 | time: 11.73270058631897s | valid accuracy 0.4717703349282297 | valid loss 1.0696563709328193
-----------------------------------------------------------
EarlyStopping counter: 7 out of 10
0.0001
-----------------------------------------------------------
| end of epoch 71 | time: 11.762580871582031s | valid accuracy 0.4717703349282297 | valid loss 1.0696963069548133

...

Adjusting learning rate of group 0 to 5.7822e-05.
-----------------------------------------------------------
| end of epoch 52 | time: 11.667012691497803s | valid accuracy 0.4666267942583732 | valid loss 1.0789330456093067
-----------------------------------------------------------
Validation loss decreased (1.079097 --> 1.078933).  Saving model ...
5.7821723252011606e-05
Adjusting learning rate of group 0 to 5.6267e-05.
-----------------------------------------------------------
| end of epoch 53 | time: 11.798157930374146s | valid accuracy 0.4666267942583732 | valid loss 1.078776196214079
-----------------------------------------------------------
Validation loss decreased (1.078933 --> 1.078776).  Saving model ...
5.6266661678215284e-05
Adjusting learning rate of group 0 to 5.4705e-05.
-----------------------------------------------------------
| end of epoch 54 | time: 11.758021354675293s | valid accuracy 0.4666267942583732 | valid loss 1.0786264072392733
-------------------------

Adjusting learning rate of group 0 to 2.4548e-05.
-----------------------------------------------------------
| end of epoch 74 | time: 11.630814552307129s | valid accuracy 0.4666267942583732 | valid loss 1.0767629057851456
-----------------------------------------------------------
Validation loss decreased (1.076816 --> 1.076763).  Saving model ...
2.4547929212481466e-05
Adjusting learning rate of group 0 to 2.3209e-05.
-----------------------------------------------------------
| end of epoch 75 | time: 11.648678541183472s | valid accuracy 0.4666267942583732 | valid loss 1.0767128333335614
-----------------------------------------------------------
Validation loss decreased (1.076763 --> 1.076713).  Saving model ...
2.320866025105019e-05
Adjusting learning rate of group 0 to 2.1896e-05.
-----------------------------------------------------------
| end of epoch 76 | time: 11.754846572875977s | valid accuracy 0.4666267942583732 | valid loss 1.0766658102738038
-------------------------

Adjusting learning rate of group 0 to 2.9560e-06.
-----------------------------------------------------------
| end of epoch 96 | time: 11.732675075531006s | valid accuracy 0.4666267942583732 | valid loss 1.0762158491229283
-----------------------------------------------------------
Validation loss decreased (1.076222 --> 1.076216).  Saving model ...
2.9559615522887324e-06
Adjusting learning rate of group 0 to 2.4472e-06.
-----------------------------------------------------------
| end of epoch 97 | time: 11.648234844207764s | valid accuracy 0.4666267942583732 | valid loss 1.0762104066728635
-----------------------------------------------------------
Validation loss decreased (1.076216 --> 1.076210).  Saving model ...
2.4471741852423276e-06
Adjusting learning rate of group 0 to 1.9853e-06.
-----------------------------------------------------------
| end of epoch 98 | time: 11.674604415893555s | valid accuracy 0.4666267942583732 | valid loss 1.0762060091240715
------------------------

-----------------------------------------------------------
| end of epoch 17 | time: 11.7051260471344s | valid accuracy 0.2686602870813397 | valid loss 1.1010381492949624
-----------------------------------------------------------
Validation loss decreased (1.101706 --> 1.101038).  Saving model ...
9.455032620941843e-05
-----------------------------------------------------------
| end of epoch 18 | time: 11.857702016830444s | valid accuracy 0.27380382775119616 | valid loss 1.1004243663704123
-----------------------------------------------------------
Validation loss decreased (1.101038 --> 1.100424).  Saving model ...
9.455032620941843e-05
-----------------------------------------------------------
| end of epoch 19 | time: 11.735990047454834s | valid accuracy 0.276555023923445 | valid loss 1.0998609598356348
-----------------------------------------------------------
Validation loss decreased (1.100424 --> 1.099861).  Saving model ...
9.455032620941843e-05
---------------------------

Adjusting learning rate of group 0 to 8.4227e-05.
-----------------------------------------------------------
| end of epoch 42 | time: 11.659666299819946s | valid accuracy 0.34760765550239237 | valid loss 1.0956630074340878
-----------------------------------------------------------
EarlyStopping counter: 4 out of 10
8.422735529643448e-05
Adjusting learning rate of group 0 to 8.3066e-05.
-----------------------------------------------------------
| end of epoch 43 | time: 11.606998205184937s | valid accuracy 0.34760765550239237 | valid loss 1.0957129049392147
-----------------------------------------------------------
EarlyStopping counter: 5 out of 10
8.306559326618263e-05
Adjusting learning rate of group 0 to 8.1871e-05.
-----------------------------------------------------------
| end of epoch 44 | time: 11.598143577575684s | valid accuracy 0.3473684210526316 | valid loss 1.0957729707237418
-----------------------------------------------------------
EarlyStopping counter: 6 out of 

## f1-score의 fold별 평균이 가장 좋았던 모델로 Test set 성능 측정
hidden state = 20 , batch = 32, lr = 0.0001, mean_f1score 

In [25]:
scale_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'BTC_close', 'DXY', 'BTCD',
       'Kimchi_premium', 'S&P500', 'Ethereum DeFi', 'News_freq']
scaler = StandardScaler()
scaler.fit(tr_set[scale_cols])#for문을 돌면서 각 train, val을 스케일링
tr_feature = scaler.transform(tr_set[scale_cols])
tr_feature = pd.DataFrame(tr_feature)                                 
test_feature = scaler.transform(test_set[scale_cols])
test_feature = pd.DataFrame(test_feature)   


test_feature, test_label = make_dataset(test_feature, test_set['signal'], window_size)


#스케일링된 데이터를 torch Tensor로 변경
test_feature = torch.FloatTensor(test_feature).to(device)
test_label  = torch.LongTensor(test_label).to(device)

final_test_set = TensorDataset(test_feature,test_label)


#데이터로더 사용
BATCH_SIZE = 32
test_dataloader = DataLoader(final_test_set , batch_size=BATCH_SIZE,shuffle=False)


In [26]:


scale_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'BTC_close', 'DXY', 'BTCD',
       'Kimchi_premium', 'S&P500', 'Ethereum DeFi', 'News_freq']



   
scaler = StandardScaler()
scaler.fit(tr_set[scale_cols])#for문을 돌면서 각 train, val을 스케일링ㅇ
train_feature = scaler.transform(tr_set[scale_cols])
train_feature = pd.DataFrame(train_feature)                                 
 

    
train_feature, train_label = make_dataset(train_feature, tr_set['signal'], window_size)
  

train_feature = torch.FloatTensor(train_feature).to(device)
train_label  = torch.LongTensor(train_label).to(device)

train_set = TensorDataset(train_feature,train_label)


#데이터로더 사용
BATCH_SIZE = 32
final_dataloader = DataLoader(train_set , batch_size=BATCH_SIZE,shuffle=False)




        
    

   

In [27]:
#모델 저장 경로
path = f"/home/jbj4278/ETH_data/biLSTM_final_model.pt"
    

In [28]:
# parameters
input_size = 12
hidden_size = 20
num_layers = 1
num_classes = 3
lr = 0.0001



In [29]:

model = BiLSTM(input_size,hidden_size, num_layers,num_classes).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0,verbose=True)
total_accu = None

Adjusting learning rate of group 0 to 1.0000e-04.


In [30]:
import time

def train(dataloader):
 
    model.train()
    total_acc, total_count = 0, 0
    start_time = time.time()

    
    loss_list = []
    for idx, (feature, label) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(feature.to(device))
        loss = criterion(predicted_label, label.to(device))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        
        total_count += label.size(0)
        loss_list.append(loss.item())
        
        if idx+1 % len(feature) == 0 and idx > 0:
            accuracy = total_acc/total_count
            loss_hist = np.mean(loss_list)
            elapsed = time.time() - start_time
            print(f'| epoch {epoch} | {idx}/{len(dataloader)} batches '
                f'| accuracy {accuracy} | loss {loss_hist}')
            total_acc, total_count = 0, 0
            start_time = time.time()

      
    return  total_acc/total_count,np.mean(loss_list)
    
            

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    loss_list = []
    pred_label = []
    pred_prob = []
    score,precision,recall = 0, 0, 0
    with torch.no_grad():
        for idx, (feature, label) in enumerate(dataloader):
            feature, label = feature.to(device),label.to(device)
            predicted_label = model(feature.to(device))
            loss = criterion(predicted_label, label.to(device))
            
            
            sm = nn.Softmax(dim=1)
            probabilities = sm(predicted_label)
            prob_arr = (probabilities.detach().cpu().numpy())[0]
            pred_prob.append(prob_arr)
            
            
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            loss_list.append(loss.item())
            pred_label.append(predicted_label.argmax(1))
            
            
            score += f1_score(predicted_label.argmax(1).to(device), label,num_classes=3)
            precision += precision_recall(predicted_label.argmax(1).to(device), label, average='macro', num_classes=3)[0]
            recall += precision_recall(predicted_label.argmax(1).to(device), label, average='macro', num_classes=3)[1]
      
        
        score = score / len(loss_list)
        precision = precision / len(loss_list)
        recall = recall / len(loss_list)
            
    
    
    return total_acc/total_count, np.mean(loss_list),score,pred_label,pred_prob


            
   

## 모델 학습

In [33]:
n_epochs = 100
early_stopping = EarlyStopping(path,patience = 3, verbose = True)


for epoch in range(1, n_epochs + 1):
    # 현재 learning rate 출력
    print(optimizer.param_groups[0]['lr'])
    epoch_start_time = time.time()
    accu_train, train_loss = train(final_dataloader)
    accu_test,test_loss,score,label,prob = evaluate(test_dataloader)
    if total_accu is not None and total_accu > accu_test:
        scheduler.step()
    else:
        total_accu = accu_test
    print('-' * 59)
    print(f'| end of epoch {epoch} | time: {time.time()- epoch_start_time}s | '
        f'valid accuracy {accu_test} | valid loss {test_loss}'
        ) 

    print('-' * 59)

    early_stopping(test_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break
print(score)
torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),


                }, path)
    

0.0001
-----------------------------------------------------------
| end of epoch 1 | time: 55.16508150100708s | valid accuracy 0.3486282382181436 | valid loss 1.0983669009959662
-----------------------------------------------------------
Validation loss decreased (inf --> 1.098367).  Saving model ...
0.0001
-----------------------------------------------------------
| end of epoch 2 | time: 55.447038888931274s | valid accuracy 0.35008125418220054 | valid loss 1.0978594035365894
-----------------------------------------------------------
Validation loss decreased (1.098367 --> 1.097859).  Saving model ...
0.0001
Adjusting learning rate of group 0 to 9.9975e-05.
-----------------------------------------------------------
| end of epoch 3 | time: 55.35290479660034s | valid accuracy 0.3499952203422235 | valid loss 1.0973830163296574
-----------------------------------------------------------
Validation loss decreased (1.097859 --> 1.097383).  Saving model ...
9.997532801828658e-05
-------

In [34]:
wandb.init(project='result', entity='gyeongmoCho',name='Bilstm')
wandb.log({'final_f1_score':score})

wandb: Currently logged in as: gyeongmocho. Use `wandb login --relogin` to force relogin


### 예측 라벨 붙여주기

In [35]:
pred_label = list(map(list, label))

answer = sum(pred_label, [])

final_label = []
for i in answer:
    final_label.append(i.tolist())

#### voting을 위한 확률값 생성

In [36]:
def make_result(model,test_dataloader):
    model.eval()
    pred_soft = []
    pred_hard=[]
    
    correct_count, all_count = 0,0
    with torch.no_grad():
        for idx, (feature, label) in enumerate(test_dataloader):
            if torch.cuda.is_available():
                feature, labels = feature.to(device),label.to(device)
            for i in range(len(labels)):
                #soft
                predicted_label = model(feature.to(device))
                sm = nn.Softmax(dim=1)
                probabilities = sm(predicted_label)
                prob_arr = (probabilities.detach().cpu().numpy())[0]
                pred_soft.append(prob_arr)
                #hard
                logps = model(feature.to(device))
                ps = torch.exp(logps)
                probab = list(ps.cpu()[0])
                pred_label = probab.index(max(probab))
                true_label = labels.cpu()[i]
                pred_hard.append(pred_label)
                if(true_label == pred_label):
                    correct_count += 1
                all_count += 1
                
                
    return pred_soft ,pred_hard




In [37]:
soft, hard = make_result(model,test_dataloader)

In [38]:
soft = pd.DataFrame(soft)

In [39]:
final_labels = pd.DataFrame(final_label,columns = ['pred_labels'])

test_set = test_set[:-10]

index = final_labels.index

test_set.index = index

plus_label_df = test_set.join(final_labels['pred_labels'],how='right') 

plus_label_df = plus_label_df.join(soft,how='right') 




In [42]:
plus_label_df.to_csv('final_BiLSTM_label.csv')

In [40]:
plus_label_df

,Date,Open,High,Low,Close,Volume,BTC_close,DXY,BTCD,Kimchi_premium,S&P500,Ethereum DeFi,News_freq,signal,pred_labels,0,1,2
0,2022-01-09 20:55:00,3934000,3934000,3931000,3933000,7.091395,52343000.0,95.739,40.435605,-12.94548,4677.04,1.401528e+11,26.0,1.0,1,0.267247,0.387458,0.345295
1,2022-01-09 20:56:00,3933000,3935000,3931000,3932000,14.144101,52315000.0,95.739,40.435605,-12.94548,4677.04,1.401528e+11,26.0,1.0,1,0.267247,0.387458,0.345295
2,2022-01-09 20:57:00,3932000,3934000,3930000,3933000,19.051929,52313000.0,95.739,40.435605,-12.94548,4677.04,1.401528e+11,26.0,1.0,1,0.267247,0.387458,0.345295
3,2022-01-09 20:58:00,3933000,3935000,3930000,3935000,11.908381,52313000.0,95.739,40.435605,-12.94548,4677.04,1.401528e+11,26.0,1.0,1,0.267247,0.387458,0.345295
4,2022-01-09 20:59:00,3935000,3936000,3934000,3935000,7.690722,52292000.0,95.739,40.435605,-12.94548,4677.04,1.401528e+11,26.0,1.0,1,0.267247,0.387458,0.345295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104605,2022-03-23 15:05:00,3662000,3664000,3660000,3662000,29.579468,51697000.0,98.793,42.584513,15.24980,4488.22,1.159212e+11,57.0,2.0,2,0.308950,0.347495,0.343555
104606,2022-03-23 15:06:00,3662000,3666000,3661000,3664000,31.708145,51710000.0,98.793,42.584513,15.24980,4488.22,1.159212e+11,57.0,2.0,2,0.308950,0.347495,0.343555
104607,2022-03-23 15:07:00,3665000,3669000,3664000,3666000,42.205886,51745000.0,98.793,42.584513,15.24980,4488.22,1.159212e+11,57.0,2.0,2,0.308950,0.347495,0.343555
104608,2022-03-23 15:08:00,3664000,3666000,3663000,3665000,20.221637,51731000.0,98.793,42.584513,15.24980,4488.22,1.159212e+11,57.0,2.0,2,0.322463,0.332964,0.344573


In [41]:
plus_label_df['pred_labels'].value_counts()


2    43450
0    39715
1    21445
Name: pred_labels, dtype: int64